In [53]:
import pandas as pd
import pyxlsb
from datetime import date, timedelta
import numpy as np

In [ ]:
estoque_dtype = {
    'Código Produto':int,
    'Código Estabelecimento':int,
    'Total': 'int64',
    'Data':object
}


estoque = pd.read_excel("Estoque_Por_Dia.xlsx",
usecols=['Código Produto','Código Estabelecimento','Total','Data'],
parse_dates=['Data'],
dtype=estoque_dtype)


estoque.rename(columns={'Total':'Estoque'},inplace=True)



In [ ]:
estoque['Data'] = pd.to_datetime(estoque['Data'],dayfirst=True)
estoque.head(10)

,Código Produto,Código Estabelecimento,Estoque,Data
0,4110020335,804,1,2025-07-08
1,4110020335,806,1,2025-07-08
2,4110020335,804,1,2025-07-09
3,4110020335,806,1,2025-07-09
4,4110020335,804,1,2025-07-10
5,4110020335,806,1,2025-07-10
6,4110020335,804,1,2025-07-11
7,4110020335,806,1,2025-07-11
8,4110020335,804,1,2025-07-12
9,4110020335,806,1,2025-07-12


In [ ]:
Vendas_dtype = {
    'Código Produto':int,
    'Código Estabelecimento':int,
    'Quantidade Faturada': 'int64',
    'Fat Bruto':float,
    'Dia do Mês':object
}



Vendas = pd.read_excel('Cubo de Vendas.xlsx',skiprows=9,
usecols=['Código Produto','Código Estabelecimento','Quantidade Faturada','Fat Bruto','Dia do Mês'],
parse_dates=['Dia do Mês'],
dtype=Vendas_dtype)

Vendas.rename(columns={'Dia do Mês':'Data'},inplace=True)

Vendas['Data'] = pd.to_datetime(Vendas['Data'],dayfirst=True)

In [ ]:
#Vendas.head(10)

In [ ]:

data_hoje = date.today()
dias_atras = 60
data_inicio = data_hoje - timedelta(days=dias_atras - 1)
df_datas = pd.DataFrame(
    pd.date_range(start=data_inicio, end=data_hoje)
)
df_datas.rename(columns={0: 'Data'}, inplace=True)

print(df_datas.head(1))
print(df_datas.tail(1))

        Data
0 2025-07-09
         Data
59 2025-09-06


In [ ]:
df_produtos_lojas = pd.concat([estoque[['Código Produto', 'Código Estabelecimento']], Vendas[['Código Produto', 'Código Estabelecimento']]]).drop_duplicates()

df_base_completa = pd.merge(df_datas, df_produtos_lojas, how='cross')


df_final = pd.merge(df_base_completa, estoque, on=['Data', 'Código Produto', 'Código Estabelecimento'], how='left')

df_final = pd.merge(df_final, Vendas, on=['Data', 'Código Produto', 'Código Estabelecimento'], how='left')

df_final = df_final.fillna(0)

df_final.sort_values(by=['Código Estabelecimento', 'Código Produto', 'Data'], inplace=True)
df_final.reset_index(drop=True, inplace=True)


             Data  Código Produto  Código Estabelecimento  Estoque  \
0      2025-07-09      4110850152                     800    136.0   
1      2025-07-10      4110850152                     800    136.0   
2      2025-07-11      4110850152                     800    136.0   
3      2025-07-12      4110850152                     800    135.0   
4      2025-07-13      4110850152                     800    135.0   
...           ...             ...                     ...      ...   
940135 2025-09-02      4336161988                     821      8.0   
940136 2025-09-03      4336161988                     821      8.0   
940137 2025-09-04      4336161988                     821      8.0   
940138 2025-09-05      4336161988                     821      8.0   
940139 2025-09-06      4336161988                     821      8.0   

        Quantidade Faturada  Fat Bruto  
0                       0.0       0.00  
1                       0.0       0.00  
2                       0.0       0.

In [ ]:
df_final.head(10)
df_final.to_excel('teste.xlsx')

In [ ]:
df_rupturas = df_final[(df_final['Quantidade Faturada'] > 0) & (df_final['Estoque'] == 0)]

df_rupturas.head(10)

,Data,Código Produto,Código Estabelecimento,Estoque,Quantidade Faturada,Fat Bruto
206,2025-08-04,4110853666,800,0.0,1.0,41.99
303,2025-07-12,4111110281,800,0.0,1.0,449.99
373,2025-07-22,4111110294,800,0.0,1.0,6299.99
399,2025-08-17,4111110294,800,0.0,1.0,5949.99
646,2025-08-24,4111110311,800,0.0,1.0,499.99
663,2025-07-12,4111110313,800,0.0,1.0,559.99
739,2025-07-28,4111110314,800,0.0,1.0,479.99
907,2025-07-16,4111110328,800,0.0,1.0,559.99
978,2025-07-27,4111110329,800,0.0,1.0,479.99
1047,2025-08-05,4111110330,800,0.0,2.0,1519.98


In [ ]:

df_final_ajustado = df_final.copy()

df_final_ajustado.loc[df_final_ajustado['Estoque'] < 0, 'Estoque'] = 0

condicao = (df_final_ajustado['Quantidade Faturada'] > 0) & (df_final_ajustado['Estoque'] == 0)

df_final_ajustado.loc[condicao, 'Estoque'] = df_final_ajustado['Quantidade Faturada']

df_final_ajustado.to_excel('teste_2.xlsx')

In [ ]:
def realizar_analise(df, group_cols):
    # a. Contagem de dias com estoque > 0 e estoque médio
    df_analise = df[df['Estoque'] > 0].groupby(group_cols).agg(
        dias_com_estoque=('Data', 'count'),
        estoque_medio=('Estoque', 'mean'),
        venda_media=('Quantidade Faturada', 'mean'),
        desvio_padrao=('Quantidade Faturada', 'std'),
        quantidade_faturada_total=('Quantidade Faturada', 'sum') 
    ).reset_index()

    df_abc = df.groupby(group_cols)['Fat Bruto'].sum().reset_index()
    df_abc.sort_values(by='Fat Bruto', ascending=False, inplace=True)
    df_abc['fat_acumulado'] = df_abc['Fat Bruto'].cumsum()
    df_abc['%_acumulado'] = df_abc['fat_acumulado'] / df_abc['Fat Bruto'].sum()
    
    def classificar_abc(porcentagem):
        if porcentagem <= 0.70:
            return 'A'
        elif porcentagem <= 0.95:
            return 'B'
        else:
            return 'C'
    
    df_abc['Classificacao_ABC'] = df_abc['%_acumulado'].apply(classificar_abc)

    # c. Junção dos resultados e cálculo do CV
    df_resultados = pd.merge(df_analise, df_abc, on=group_cols, how='left')

    # Calcula o CV, tratando a divisão por zero
    df_resultados['CV'] = np.where(df_resultados['venda_media'] > 0, 
                                 df_resultados['desvio_padrao'] / df_resultados['venda_media'], 
                                 np.nan)

    return df_resultados



In [ ]:
analise_por_loja = realizar_analise(df_final_ajustado, ['Código Estabelecimento', 'Código Produto'])

df_geral_consolidado = df_final_ajustado.groupby(['Data', 'Código Produto']).agg(
    Estoque=('Estoque', 'sum'),
    Quantidade_Faturada=('Quantidade Faturada', 'sum'),
    Fat_Bruto=('Fat Bruto', 'sum')
).reset_index()

analise_geral = realizar_analise(df_geral_consolidado, ['Código Produto'])


KeyError: "Column(s) ['Quantidade Faturada'] do not exist"

In [ ]:

with pd.ExcelWriter('analise_completa.xlsx', engine='xlsxwriter') as writer:
    analise_por_loja.to_excel(writer, sheet_name='Análise por Loja', index=False)
    analise_geral.to_excel(writer, sheet_name='Análise Geral', index=False)

print("Análises completas geradas com sucesso no arquivo 'analise_completa.xlsx'!")


Análises completas geradas com sucesso no arquivo 'analise_completa.xlsx'!


Note: you may need to restart the kernel to use updated packages.
